# Lab 3 - The Matrix Form and The Duality Theory

<b>Information on group members:</b><br>
1) 156071, Martyna Stasiak <br>
2) 156062, Maria Musiał

In [2]:
from pulp import *  
import numpy as np
import pandas as pd
import itertools
from IPython.display import display

# 1) The Matrix Form - Fundamental Insight (finish this part to get 3.0)

1.1) Given is the below (primal) linear problem:

primal problem:<br>
MAXIMIZE<br>
4*x1 + 2*x2 + 3*x3 <br>

SUBJECT TO<br>
1_constraint: x1 + x2 <= 10<br>
2_constraint: 2*x2 + x3 <= 12<br>
3_constraint: 3*x1 + 2*x3 <= 15<br>
4_constraint: x1 + x2 + x3 <= 20<br>

VARIABLES<br>
x1 Continuous<br>
x2 Continuous<br>
x3 Continuous<br>

1.2) Use the PuLP library to solve the above problem. Identify the optimal solution: the values for basic variables and the corresponding value for the objective function.

In [3]:
### Model here:
model = LpProblem("Lab3TaskA", LpMaximize)
# Variables
x1 = LpVariable(name="x1", lowBound=0) #do they have to be nonnegative??????????
x2 = LpVariable(name="x2", lowBound=0)
x3 = LpVariable(name="x3", lowBound=0)

# Constraints
# x1 + x2 <= 10
model += (x1 + x2 <= 10, "#1 constraint")
# 2*x2 + x3 <= 12
model += (2*x2 + x3 <= 12, "#2 constraint")
# 3*x1 + 2*x3 <= 15
model += (3*x1 + 2*x3 <= 15, "#3 constraint")
# x1 + x2 + x3 <= 20
model += (x1 + x2 + x3 <= 20, "#4 constraint")

# Objective
obj_func = 4*x1 + 2*x2 + 3*x3
model += obj_func

model


Lab3TaskA:
MAXIMIZE
4*x1 + 2*x2 + 3*x3 + 0
SUBJECT TO
#1_constraint: x1 + x2 <= 10

#2_constraint: 2 x2 + x3 <= 12

#3_constraint: 3 x1 + 2 x3 <= 15

#4_constraint: x1 + x2 + x3 <= 20

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous

In [4]:
### Solve here
status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}\n")
print('Constraints:')
for name, constraint in model.constraints.items():  print(f"{name}: {constraint.value()}")
print('\nVariables & their values:')
for var in model.variables():  print(f"{var.name}: {var.value()}")

status: 1, Optimal
objective: 31.428571379999998

Constraints:
#1_constraint: 0.0
#2_constraint: 5.999999985739635e-08
#3_constraint: -7.999999906971311e-08
#4_constraint: -9.14285714

Variables & their values:
x1: 4.4285714
x2: 5.5714286
x3: 0.85714286


1.3) In this exercise, you are asked to identify ALL basic (feasible and not) solutions to the above problem. We will do this naively. Specifically, you are asked to use the fundamental insight to build a final simplex tableau for each possible base. Therefore, you need first to initialize the data: c, b, A matrixes, and it is suggested to initialize the auxiliary matrix M defined as M = A + (concatenate) I (identity matrix). Note that the problem should be formulated in the augmented form. Then, you have to iterate over each possible base B, compute B-1, and other relevant parts for the simplex tableau.<br><br>
a) Identify the optimal solution using the optimality condition; print it (Z value and values for basic variables); compare thus derived solution with the optimum found using the PuLP library (obviously, both solutions should be the same). <br>
b) Count the number of feasible and infeasible solutions. How many (all) basic solutions to the problem can be identified? <br><br>
It is suggested to use the NumPy library for performing matrix operations. 

### Augmented form:
MAXIMIZE<br>
Z -4*x1 - 2*x2 - 3*x3 <br>

SUBJECT TO<br>
1) x1 + x2 + x4 = 10<br>
2) 2*x2 + x3 + x5 = 12<br>
3) 3*x1 + 2*x3 + x6 = 15<br>
4) x1 + x2 + x3 + x7 = 20<br>

In [11]:
I = np.eye(4)
II = np.eye(A.shape[0])# identity matrix
I == I


array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [ ]:
### Initialize the data here:
### TODO
c = np.array([4, 2, 3]) #coefficients in the objective function
b = np.array([10, 12, 15, 20]).transpose() #right-hand side of the constraints
A = np.array([[1, 1, 0], [0, 2, 1], [3, 0, 2], [1, 1, 1]]) #coefficients in the constraints
I = np.eye(4) # identity matrix
M = np.concatenate((A, I), axis=1) # The matrix M
base = np.array([3, 4, 5, 6]) # The base corresponds to the indices of the identity matrix columns in M
c_extended = np.concatenate((c, np.zeros(4))) # The extended objective function coefficients


<b> Important note: the below is just a proposition. You can solve the problem in your own way. </b>

You can define an auxiliary method constructing a final simplex tableau for a given base.  Here, "base" is a list of columns (integers) for the base. Note that the functions in python can return multiple objects and you can use this functionality to return<br>
- the inversed base<br>
- coefficients in the row 0 for slack variables<br>
- right side values (except the objective function value)<br>
- the objective function value<br>
- the coefficients for decision variables in row 0 <br>
- the coefficients for decision variables in rows 1+<br>

Note that if BI cannot be built (it is possible), the method may return None in order to notify the executive method about this exception. 



In [ ]:
def getFinalTableau(base, c, b, A, M):
    # TODO
    # return None

In [21]:
# Define the pivot function, which is a core operation in the Simplex algorithm
def pivot(tableau, row, col):
    # Make the pivot element equal to 1 by dividing the row by the pivot element
    tableau[row, :] /= tableau[row, col]
    
    # Make all other entries in the pivot column zero
    for i in range(tableau.shape[0]):
        if i != row:
            tableau[i, :] -= tableau[i, col] * tableau[row, :]
    
    # Update the base array to reflect the new entering and leaving variable indices
    base[row - 1] = col  # Adjust index since row is offset by the objective row

# Define the Simplex algorithm function that iteratively improves the solution
def simplex(tableau, base):
    while True:
        # Find the entering variable (the most negative value in the objective row)
        col = np.argmin(tableau[0, :-1])  # Objective row is the first row
        if tableau[0, col] >= 0:
            # If all values in the objective row are non-negative, the solution is optimal
            break

        # Find the leaving variable (row with the smallest positive ratio)
        # Use rows 1 onward, ignoring the objective row, for the ratio test
        ratios = tableau[1:, -1] / tableau[1:, col]
        
        # Only consider positive entries in the pivot column for valid ratios
        valid_ratios = np.where(tableau[1:, col] > 0, ratios, np.inf)
        row = np.argmin(valid_ratios) + 1  # Offset by 1 to account for the objective row
        
        # Perform the pivot operation on the tableau, updating for the new basis variable
        pivot(tableau, row, col)

    # Return the modified tableau and the final base array indicating which variables are basic
    return tableau, base

# Define getFinalTableau to return a DataFrame for easy interpretation of the final tableau
def getFinalTableau(base, c, b, A, M):
    # Extend c with zeros for the slack variables to match the tableau width
    c_extended = np.concatenate((c, np.zeros(M.shape[1] - c.size)))
    
    # Create the initial tableau:
    # - Objective function row is placed at the top
    # - Stack the constraint rows with RHS values `b`
    tableau = np.vstack([np.hstack([-c_extended, [0]]), np.hstack([M, b.reshape(-1, 1)])])
    
    # Run the Simplex algorithm to optimize the tableau
    final_tableau, final_base = simplex(tableau, base)

    # Define column names for the tableau
    num_vars = c.size
    num_slack = M.shape[1] - num_vars
    column_names = [f"x{i+1}" for i in range(num_vars)] + [f"s{i+1}" for i in range(num_slack)] + ["RHS"]

    # Define row names for each row of the tableau:
    # - First row is labeled "Objective"
    # - For basic variables, use the current values in the `base` array
    row_names = ["Objective"] + [f"s{int(i - num_vars + 1)}" if i >= num_vars else f"x{i+1}" for i in final_base]

    # Convert the final tableau to a DataFrame for clearer visualization
    tableau_df = pd.DataFrame(final_tableau, index=row_names, columns=column_names)
    return tableau_df

# Initialize the data for the linear program
c = np.array([4, 2, 3])  # Coefficients of the original variables in the objective function
b = np.array([10, 12, 15, 20])  # Right-hand side values of each constraint
A = np.array([[1, 1, 0], [0, 2, 1], [3, 0, 2], [1, 1, 1]])  # Coefficients for the constraints
I = np.eye(4)  # Identity matrix to add slack variables, representing slack columns

# Combine A and I matrices to form M, the full constraint matrix including slack variables
M = np.concatenate((A, I), axis=1)

# Initialize the base array with indices of the slack variables
# - Slack variables correspond to columns of the identity matrix in M, so base starts with [3, 4, 5, 6]
base = np.array([3, 4, 5, 6])

# Get the final tableau as a DataFrame after running the Simplex algorithm
final_tableau_df = getFinalTableau(base, c, b, A, M)

# Print the final tableau in DataFrame format for easy interpretation
print("Final tableau:\n", final_tableau_df)


Final tableau:
             x1   x2   x3        s1        s2        s3   s4        RHS
Objective  0.0  0.0  0.0  0.571429  0.714286  1.142857  0.0  31.428571
x2         0.0  1.0  0.0  0.428571  0.285714 -0.142857  0.0   5.571429
x3         0.0  0.0  1.0 -0.857143  0.428571  0.285714  0.0   0.857143
x1         1.0  0.0  0.0  0.571429 -0.285714  0.142857  0.0   4.428571
s4         0.0  0.0  0.0 -0.142857 -0.428571 -0.285714  1.0   9.142857


C:\Users\mmart\AppData\Local\Temp\ipykernel_19508\3201420104.py:25: RuntimeWarning: divide by zero encountered in divide
  ratios = tableau[1:, -1] / tableau[1:, col]


In [22]:
import numpy as np
import pandas as pd

# Define the pivot function, which updates the tableau based on the selected pivot row and column
def pivot(tableau, row, col):
    # Make the pivot element equal to 1 by dividing the row by the pivot element
    tableau[row, :] /= tableau[row, col]
    
    # Make all other entries in the pivot column zero
    for i in range(tableau.shape[0]):
        if i != row:
            tableau[i, :] -= tableau[i, col] * tableau[row, :]
    
    # Update the base array to reflect the new entering and leaving variable indices
    base[row - 1] = col  # Offset by -1 since base doesn't include the objective row

# Define the Simplex algorithm function that iteratively improves the solution
def simplex(tableau, base):
    feasible_count = 0  # Counter for feasible solutions
    infeasible_count = 0  # Counter for infeasible solutions

    while True:
        # Check for optimality by finding the most negative entry in the objective row
        col = np.argmin(tableau[0, :-1])
        if tableau[0, col] >= 0:
            # If no negative values, the solution is optimal
            break

        # Identify the leaving variable using the minimum ratio test on RHS values
        ratios = tableau[1:, -1] / tableau[1:, col]
        valid_ratios = np.where(tableau[1:, col] > 0, ratios, np.inf)
        row = np.argmin(valid_ratios) + 1  # Offset by +1 to account for objective row

        # Count feasible and infeasible solutions based on RHS values
        if tableau[row, -1] >= 0:
            feasible_count += 1
        else:
            infeasible_count += 1

        # Perform the pivot operation on the tableau and update the base array
        pivot(tableau, row, col)

    # Return the modified tableau, final base, and counts of feasible/infeasible solutions
    return tableau, base, feasible_count, infeasible_count

# Define getFinalTableau to return a DataFrame and additional information
def getFinalTableau(base, c, b, A, M):
    # Extend c with zeros for slack variables to match the tableau width
    c_extended = np.concatenate((c, np.zeros(M.shape[1] - c.size)))
    
    # Create the initial tableau with objective row at the top and constraints below
    tableau = np.vstack([np.hstack([-c_extended, [0]]), np.hstack([M, b.reshape(-1, 1)])])
    
    # Run the Simplex algorithm to get the final tableau, base, and feasibility counts
    final_tableau, final_base, feasible_count, infeasible_count = simplex(tableau, base)

    # Define column names for the tableau
    num_vars = c.size
    num_slack = M.shape[1] - num_vars
    column_names = [f"x{i+1}" for i in range(num_vars)] + [f"s{i+1}" for i in range(num_slack)] + ["RHS"]

    # Define row names: first row is "Objective"; others use the base array to label basic variables
    row_names = ["Objective"] + [f"s{int(i - num_vars + 1)}" if i >= num_vars else f"x{i+1}" for i in final_base]

    # Convert final tableau to DataFrame for clearer visualization
    tableau_df = pd.DataFrame(final_tableau, index=row_names, columns=column_names)

    # Extract the values of basic variables from the RHS column in the final tableau
    basic_vars = {row_names[i + 1]: final_tableau[i + 1, -1] for i in range(len(final_base))}
    objective_value = final_tableau[0, -1]

    # Print results
    print("Final Tableau:\n", tableau_df)
    print("\nBasic Variables and Their Values:")
    for var, value in basic_vars.items():
        print(f"{var} = {value}")
    print(f"\nOptimal Objective Function Value: {objective_value}")
    print(f"\nFeasible solutions count: {feasible_count}")
    print(f"Infeasible solutions count: {infeasible_count}")
    print(f"Total solutions count: {feasible_count + infeasible_count}")

    return tableau_df, basic_vars, objective_value, feasible_count, infeasible_count

# Initialize the data for the linear program
c = np.array([4, 2, 3])  # Coefficients of the original variables in the objective function
b = np.array([10, 12, 15, 20])  # Right-hand side values of each constraint
A = np.array([[1, 1, 0], [0, 2, 1], [3, 0, 2], [1, 1, 1]])  # Coefficients for the constraints
I = np.eye(4)  # Identity matrix to add slack variables, representing slack columns

# Combine A and I matrices to form M, the full constraint matrix including slack variables
M = np.concatenate((A, I), axis=1)

# Initialize the base array with indices of the slack variables
# - Slack variables correspond to columns of the identity matrix in M, so base starts with [3, 4, 5, 6]
base = np.array([3, 4, 5, 6])

# Get the final tableau as a DataFrame after running the Simplex algorithm
final_tableau_df, basic_vars, objective_value, feasible_count, infeasible_count = getFinalTableau(base, c, b, A, M)


Final Tableau:
             x1   x2   x3        s1        s2        s3   s4        RHS
Objective  0.0  0.0  0.0  0.571429  0.714286  1.142857  0.0  31.428571
x2         0.0  1.0  0.0  0.428571  0.285714 -0.142857  0.0   5.571429
x3         0.0  0.0  1.0 -0.857143  0.428571  0.285714  0.0   0.857143
x1         1.0  0.0  0.0  0.571429 -0.285714  0.142857  0.0   4.428571
s4         0.0  0.0  0.0 -0.142857 -0.428571 -0.285714  1.0   9.142857

Basic Variables and Their Values:
x2 = 5.571428571428571
x3 = 0.8571428571428572
x1 = 4.428571428571429
s4 = 9.142857142857142

Optimal Objective Function Value: 31.42857142857143

Feasible solutions count: 3
Infeasible solutions count: 0
Total solutions count: 3


C:\Users\mmart\AppData\Local\Temp\ipykernel_19508\4012294565.py:30: RuntimeWarning: divide by zero encountered in divide
  ratios = tableau[1:, -1] / tableau[1:, col]


In [ ]:
def getFinalTableau(base, c, b, A, M):
    B = M[:, base] # Basis matrix
    
    if np.linalg.det(B) == 0:
        print("Basis matrix is singular so it is not invertible")
        return None
    
    B_inv = np.linalg.inv(B) # Inverse of the basis matrix
    
    basic_solution = np.dot(B_inv, b) # Basic solution
    
    cB = c[base[:len(c)]] # Objective function coefficients for the basic variables
    z = np.dot(cB, basic_solution) # Objective function value
    
    row0 = c - cB.dot(B_inv).dot(A) # Row 0 of the tableau
    
    tableau_constraints = np.dot(B_inv, M) # Tableau constraints
    
    return {
        "B_inv": B_inv,
        "basic_solution": basic_solution,
        "objective_value": z,
        "row0": row0,
        "tableau_constraints": tableau_constraints
    }
    

In [18]:
### Solve the problem here
### TODO

The base matrix B is singular and cannot be inverted.


## 2) The Duality Theory (finish this part + part 1 + to get 5.0)

2.1) Model the dual problem to the above solved primal one, using the PuLP library. Then, solve it and compare the derived optimum with the optimum for the primal problem. Are they equal?

In [ ]:
### Model here:
### TODO


In [ ]:
### Solve here:
### TODO


2.2) This exercise is based on the exercise 1.3 (copy & paste solution). Here, you are asked to iterate over all basic solutions (as in 1.3) and store them along with their complementary dual solutions. Solutions should be stored in the PRIMAL_DUAL_SOLUTIONS list and sorted according to the objective value Z = W. Analyze their optimality and feasibility. Finally, you have to display all basic solutions wlong with their complementary solutions (you can use the provided piece of code written using the pandas library). <br><br>

PRIMAL_DUAL_SOLUTIONS is defined as a table consisting of n rows, where n is the number of basic solutions to the problem, and 21 columns. The columns are defined as follows:<br>
Col. 1: The objective value Z<br>
Col. 2-4: The values for decision variables (primal solution)<br>
Col. 5-8: The values for slack variables (primal solution)<br>
Col. 9: P_F = Y or N, Y/N = primal solution is feasible/infeasible<br>
Col. 10: P_O = Y or N, Y/N = primal solution is optimal/is not optimal<br>
Col. 11: P_STATE = -/suboptimal/superoptimal/optimal; depends on P_F and P_O (primal)<br>
Col. 12: D_STATE = -/suboptimal/superoptimal/optimal; depends on D_F and D_O (dual)<br>
Col. 13: D_F = Y or N, Y/N = dual solution is feasible/infeasible<br>
Col. 14: D_O = Y or N, Y/N = dual solution is optimal/is not optimal<br>
Col. 15-18: The values for decision variables (dual solution)<br>
Col. 19-21: The values for surplus variables (dual solution)<br><br>

Reminder: sort solutions according to Z; analyze how their states change with the increase of Z.

In [ ]:
### TODO 

### DISPLAY STORED PAIRS OF SOLUTIONS
#df = pd.DataFrame(PRIMAL_DUAL_SOLUTIONS, columns = ["Z", "x1", "x2", "x3", "slack1", "slack2", "slack3", "slack4", "P_F", "P_O", "P_STATE", "D_STATE", "D_F", "F_O",
#                                                   "y1", "y2", "y3", "y4", "sur1", "sur2", "sur3"])

#display(df.style.set_properties(**{
#        'width': '230px',
#        'max-width': '230px',
#    }))